In [32]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
daily_path =  "C:\\Users\\goodluck\\Desktop\\DB\\daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

# 更换数据库地址
DTBS_path = '..\\database_storage\\DTBS.pkl'
DCBS_path = '..\\database_storage\\DCBS.pkl'

# re-open
with open(DCBS_path, 'rb') as f:  
    DCBS = pickle.load(f)
# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)
    
# 这个文件用于实时换仓，每周一/三/四换模拟仓，每周二2点半换实仓

In [33]:
def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if DCBS[lm[0]].__contains__(code):
                if low_prc(code, dt, lm, fcd[1]):
                    satisfy.append(code)
        
    else:
        for code in codes:
            if DCBS[dt][code][fcd[0]] <= fcd[2] and  DCBS[dt][code][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy


def takeSecond(elem):
    return elem[1]

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DCBS[dt][code]['dp']):
                lm = calculate_time(dt, 'p', 20)
                if DCBS[lm[0]].__contains__(code):
                    prc = get_prcs(code, lm)
                    lp = absolute_low(prc, 0)
                    rk.append((code,  DCBS[dt][code]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan( DCBS[dt][code][idx]):
                rk.append((code,  DCBS[dt][code][idx]))
        else:
            if not np.isnan( DCBS[dt][code][idx]):
                if  DCBS[dt][code][idx] != 999:
                    rk.append((code,  DCBS[dt][code][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]

def get_prcs(code, dts):
    prc = []
    
    for dt in dts:
        prc.append(DCBS[dt][code]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DCBS[dt][code]['dp']
#     if (tp <= lp):
#         print(code, tp, lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

def calculate_time(dt, d, l):
    time = []
    days = list(DCBS.keys())
    if d == 'p':
        for i in range(len(days)):
            if days[i] == dt:
                break
                
    return days[i-l+1:i+1]



In [34]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
# td取DCBS中的日期最后一天
td = sorted(list(DCBS.keys()))[-1]

print('日期:',td)
DCBS.keys()

日期: 2023-08-31


dict_keys(['2023-04-27', '2023-04-28', '2023-05-04', '2023-05-05', '2023-05-08', '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-22', '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01', '2023-06-02', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30', '2023-07-03', '2023-07-04', '2023-07-05', '2023-07-06', '2023-07-07', '2023-07-10', '2023-07-11', '2023-07-12', '2023-07-13', '2023-07-14', '2023-07-17', '2023-07-18', '2023-07-19', '2023-07-20', '2023-07-21', '2023-07-24', '2023-07-25', '2023-07-26', '2023-07-27', '2023-07-28', '2023-07-31', '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04', '2023-08-07', '2023-08-08', '2023-08-09', '2023-08-

In [40]:
select = list(DCBS[td].keys())

# print(select)

# flt_cd = [['dp', 100, 130]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]

# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]

# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]

# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]

# flt_cd = [['dp', 100, 130], ['bl',0, 3]]
# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]
# flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]

# flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]



# 高ytm，周三
flt_cd = [['dp',100,130],['ytm', 0.5, 999], ['bl', 0, 7], ['trt',2,10]]



# 双低5叠加/区间双低3叠加,实盘，这个策略是可以更改['dp',100,130]里面的参数的，周二
flt_cd = [['dp',100,130],['ytm', -1, 999], ['bl', 0, 7], ['trt',2,10]]


# 低价，周一
flt_cd = [['lp', 0.5], ['dp', 110, 130], ['bl',0, 3], ['cpr', -999, 50]]

# 小市值3双低5，周三
flt_cd = [['dp', 100, 130], ['bl',0, 3]]


for fcd in flt_cd:

    select = select_condition(select, td, fcd)




# 按照高ytm排序，用于高ytm策略的排序
rk_cd = [['ytm', 20, 'd']]

# 低价，用于低价策略排序
rk_cd = [['lp', 30, 'a']]


# 按照双低排序，用于小市值3双低5策略、双低5叠加/区间双低3叠加等三个策略排序
rk_cd = [['dl', 30, 'a']]

codes = rank(select, td, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])

In [42]:
output_information = []

# 改用display方法输出dataframe，显示效果更佳

for r in codes:
    
    last_day = list(sorted(DTBS['A'][r].keys()))[-1]
    
    # 这里的修改是由于在F区可能并没有存储某些新上市转债的信息，F区的季度信息只有2023-03
    if DTBS['B'][r]['sc'] not in DTBS['F'].keys():
        # print(r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], "价格:", DCBS[td][r]['dp'],  "转股溢价率:", DCBS[td][r]['cpr'], "YTM:", DCBS[td][r]['ytm'],"换手率:", DCBS[td][r]['trt'], '市值:',  DCBS[td][r]['bl'], '货币资金:',  'F区未存储', '双低:', round(DCBS[td][r]['dl'],2), 'qs30:',DTBS['A'][r][last_day]['qs30'],'风险提示:',DTBS['B'][r].get('risk_warning','无'))
        output_information.append([r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], DCBS[td][r]['dp'], DCBS[td][r]['cpr'],DCBS[td][r]['ytm'],DCBS[td][r]['trt'],DCBS[td][r]['bl'],'F区未存储', round(DCBS[td][r]['dl'],2),DTBS['A'][r][last_day]['qs30'], DTBS['B'][r].get('risk_warning','无')])        
    else:    
        # print(r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], "价格:", DCBS[td][r]['dp'],  "转股溢价率:", DCBS[td][r]['cpr'], "YTM:", DCBS[td][r]['ytm'],"换手率:", DCBS[td][r]['trt'], '市值:',  DCBS[td][r]['bl'], '货币资金:',  round(DTBS['F'][DTBS['B'][r]['sc']]['2023-03']['hb'],2), '双低:', round(DCBS[td][r]['dl'],2), 'qs30:',DTBS['A'][r][last_day]['qs30'],'风险提示:',DTBS['B'][r].get('risk_warning','无'))        
        output_information.append([r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], DCBS[td][r]['dp'], DCBS[td][r]['cpr'],DCBS[td][r]['ytm'],DCBS[td][r]['trt'],DCBS[td][r]['bl'],round(DTBS['F'][DTBS['B'][r]['sc']]['2023-03']['hb'],2), round(DCBS[td][r]['dl'],2),DTBS['A'][r][last_day]['qs30'], DTBS['B'][r].get('risk_warning','无')])
        
output_df = pd.DataFrame(output_information,columns=['可转债代码','可转债名称','正股代码','正股名称','2点实时价格','转股溢价率','YTM','换手率','债券余额','货币资金','双低','qs30','风险提示'])

# 显示
display(output_df)

,可转债代码,可转债名称,正股代码,正股名称,2点实时价格,转股溢价率,YTM,换手率,债券余额,货币资金,双低,qs30,风险提示
0,113595.SH,花王转债,603007.SH,ST花王,108.843,3.97,4.40,8.63,2.707,0.18,112.81,0,无
1,123002.SZ,国祯转债,300388.SZ,节能国祯,108.334,28.02,-9.25,3.38,2.521,12.77,136.35,0,无
2,128033.SZ,迪龙转债,002658.SZ,雪迪龙,119.750,17.81,-35.52,7.26,2.972,5.56,137.56,0,无
3,128021.SZ,兄弟转债,002562.SZ,兄弟科技,111.525,35.48,-20.32,2.58,2.651,3.97,147.00,0,无
4,128039.SZ,三力转债,002224.SZ,三力士,118.436,31.55,-13.64,1.41,2.024,11.49,149.99,0,无
5,113546.SH,迪贝转债,603320.SH,迪贝电气,128.590,21.92,-3.77,4.71,2.297,3.63,150.51,0,无
6,123166.SZ,蒙泰转债,300876.SZ,蒙泰高新,128.949,24.72,-1.17,3.67,3.000,2.09,153.67,0,无
7,128071.SZ,合兴转债,002228.SZ,合兴包装,115.273,39.03,-1.56,0.92,2.999,6.19,154.30,0,无
8,113527.SH,维格转债,603518.SH,锦泓集团,125.879,29.51,-5.15,2.79,2.862,6.61,155.39,0,无
9,110070.SH,凌钢转债,600231.SH,凌钢股份,120.982,39.27,-1.75,4.12,2.170,36.20,160.25,0,无


In [31]:
# 寻找双低、价格分位点
td = list(DCBS.keys())[-1]

dp_list = []
dl_list = []

for bond_code,informations in DCBS[td].items():
    dp_list.append(informations['dp'])
    dl_list.append(informations['dl'])
    
print(td, '\n',
      '价格前30%分位点:', round(np.quantile(dp_list,0.3),2),'\n',
      '价格前50%分位点:', round(np.quantile(dp_list,0.5),2),'\n',
      '双低前30%分位点:', round(np.quantile(dl_list,0.3),2),'\n',
      '双低前50%分位点:', round(np.quantile(dl_list,0.5),2))

2023-08-31 
 价格前30%分位点: 115.59 
 价格前50%分位点: 122.07 
 双低前30%分位点: 157.76 
 双低前50%分位点: 168.67


In [16]:
DCBS['2023-08-29']['118030.SH']

{'cpr': 12.8,
 'dp': 147.981,
 'dl': 160.781,
 'ytm': -4.68,
 'bl': 15.638,
 'trt': 5.21,
 'yl': 5.343,
 'csp': 39.92,
 'ia': 1,
 'ipo': '2023-02-10',
 'cn': '睿创转债',
 'sc': '688002.SH',
 'sn': '睿创微纳',
 'cat1': nan,
 'cat2': nan,
 'cl': 52.37,
 'pb': 5.18}

In [21]:
DTBS['A']['110062.SH']['2023-08-28']

{'cpr': 62.0,
 'dp': 119.68,
 'dl': 181.68,
 'ytm': -4.0,
 'bl': 30.873,
 'trt': 1.54,
 'yl': 2.266,
 'csp': 22.66,
 'ia': 1,
 'xx': 1,
 'hs': 0,
 'csv': 73.87466902030008,
 'qs': 0,
 'qs15': 0,
 'qs30': 0,
 'xx15': 15,
 'xx30': 23,
 'cz': 103.2964,
 'pd': -28.482823195871234}

In [17]:
DTBS['B']

{'110088.SH': {'cn': '淮22转债',
  'ipo': '2022-10-20',
  'sc': '600985.SH',
  'sn': '淮北矿业',
  'cat1': '煤炭',
  'cat2': '煤炭开采',
  'dld': '2046-01-01',
  'sw': '投资',
  'zs': '周期',
  'yqq': '能源',
  'xr': 85,
  'xt1': 20,
  'xt2': 30,
  'gq': 1,
  'qr': 130,
  'qt1': 15,
  'qt2': 30,
  'company_nature': '地方国有企业',
  'hot_concept': '参股保险;沪股通;煤化工;富时罗素概念;富时罗素概念股;煤炭概念;标普道琼斯A股;甲醇;融资融券;民爆概念;转融券标的;焦炭;地方国企改革;国企改革;安徽国企改革'},
 '113545.SH': {'cn': '金能转债',
  'ipo': '2019-11-07',
  'sc': '603113.SH',
  'sn': '金能科技',
  'cat1': '煤炭',
  'cat2': '焦炭Ⅱ',
  'dld': '2046-01-01',
  'sw': '投资',
  'zs': '周期',
  'yqq': '能源',
  'xr': 80,
  'xt1': 15,
  'xt2': 30,
  'gq': 0,
  'qr': 130,
  'qt1': 15,
  'qt2': 30,
  'company_nature': '民营企业',
  'hot_concept': '焦炭;煤化工;沪股通;聚丙烯;粗苯;白炭黑;氢能源;融资融券;转融券标的;富时罗素概念;富时罗素概念股;甲醇;标普道琼斯A股;煤炭概念;炭黑;光伏概念'},
 '113631.SH': {'cn': '皖天转债',
  'ipo': '2021-12-10',
  'sc': '603689.SH',
  'sn': '皖天然气',
  'cat1': '公用事业',
  'cat2': '燃气Ⅱ',
  'dld': '2046-01-01',
  'sw': '服务',
  'zs': '能源',
  'yqq': '能源'